In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# %conda install matplotlib

In [5]:
os.getcwd()
os.listdir('../all-domestic-certificates/domestic-E06000001-Hartlepool')

['certificates.csv', 'recommendations.csv']

In [6]:
#RUN
df_cert = pd.read_csv(
    '../all-domestic-certificates/domestic-E06000001-Hartlepool/certificates.csv', 
    dtype = {'INSPECTION_DATE':'str','LODGEMENT_DATE':'str'},
    parse_dates=['INSPECTION_DATE','LODGEMENT_DATE']
)
df_rec = pd.read_csv('../all-domestic-certificates/domestic-E06000001-Hartlepool/recommendations.csv')

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3888247005.py:2: DtypeWarning: Columns (34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cert = pd.read_csv(


In [7]:
df_cert.CURRENT_ENERGY_RATING.value_counts()

D    17756
C    12300
E     6289
B     3974
F     1337
G      427
A       90
Name: CURRENT_ENERGY_RATING, dtype: int64

In [8]:
len(df_cert), len(df_rec)

(42173, 177258)

## Cleansing NaNs

Quick cleanse to replace dubious labels with Nan.
Will do proper categories cleanse later.

In [9]:
#RUN
# create copy
cert = df_cert.copy() 

#lower case all col headers to speed up typing
cert.columns = [col.lower() for col in cert.columns]

# pd.Series(cert.columns).to_csv('columns.csv',index=False)

# replace 'NO DATA!', 'not defined' and 'not recored' with null
cert = cert.replace(['NO DATA!','NODATA!'],np.nan)
cert = cert.replace(['not recorded','not defined','unknown','Unknown','Not defined','Not recorded'],np.nan)
cert = cert.replace(['N/A','n/a'],np.nan)

# replace any values starting sap as these are the column headers not real data
cert = cert.replace([r'^SAP.*',r'^sap.*'],np.nan,regex=True)

# replace '+ Chr(13) +' as it's meaningless
cert = cert.replace([r"\+ Chr\(13\) \+",r"\+ chr\(13\) \+"],'+', regex=True)
cert = cert.replace([r"\+ Chr\(13\) \+",r"\+ chr\(13\) \+"],'+', regex=True)

# replacing values in columns which are actually just the column header
replacement_dict = {
    'hotwater_description':{r'^Hot-Water$':np.nan},
    'floor_description':{r'^Floor$':np.nan},
    'walls_description':{r'^Wall$':np.nan},
    'windows_description':{r'^Window$':np.nan},
    'roof_description':{r'^Roof$':np.nan},
    'main_heating_controls':{r'^Main-Heating-Controls$':np.nan},
    'secondheat_description':{r'^Secondary-Heating$':np.nan},
    'lighting_description':{r'^Lighting$':np.nan},
    'mainheatcont_description':{r'^Main-Heating-Controls$':np.nan}}

cert = cert.replace(replacement_dict,regex=True)

#### PROPERTY_TYPE

In [10]:
#RUN
#just drop the Park Homes
cert = cert.drop(cert[cert.property_type=='Park home'].index)
cert.property_type.isna().sum()

0

#### BUILT_FORM

In [11]:
#RUN
#remove 'enclosed' from the terraced properties 

cert.built_form = cert.built_form.map({
    'Enclosed End-Terrace':'End-Terrace',
    'Enclosed Mid-Terrace':'Mid-Terrace',
    'Mid-Terrace':'Mid-Terrace',
    'Semi-Detached':'Semi-Detached',
    'End-Terrace':'End-Terrace',
    'Detached':'Detached'
})

cert.built_form.value_counts()

Mid-Terrace      16150
Semi-Detached    11007
End-Terrace       7583
Detached          6849
Name: built_form, dtype: int64

#### ENERGY_TARIFF

In [12]:
#RUN
#replace the unknowns with actual NAs

cert['energy_tariff'] = cert.energy_tariff.replace('Unknown',np.nan)

cert.energy_tariff.value_counts()

Single              33402
standard tariff      3768
dual                 1928
off-peak 7 hour      1742
off-peak 10 hour       83
24 hour                53
dual (24 hour)         13
off-peak 18 hour        1
Name: energy_tariff, dtype: int64

#### GLAZED_TYPE

In [13]:
#RUN
#remap these messy categories 
cert.glazed_type = cert.glazed_type.replace(['double, known data','secondary glazing'],'double glazing, unknown install date')

cert.glazed_type = cert.glazed_type.replace(['INVALID!','not defined'],np.nan)

cert.glazed_type.value_counts()

double glazing installed before 2002             15610
double glazing installed during or after 2002    11648
double glazing, unknown install date              9963
single glazing                                     195
triple glazing                                      18
Name: glazed_type, dtype: int64

#### TENURE

In [14]:
#RUN
cert.tenure = cert.tenure.replace('unknown',np.nan)
cert.tenure = cert.tenure.replace('Not defined.*',np.nan,regex=True)
cert.tenure.value_counts()

owner-occupied      18225
rental (social)      8906
rental (private)     6090
Rented (social)      2197
Owner-occupied       1901
Rented (private)      695
Name: tenure, dtype: int64

## Impute w/in same building 
Fill in NaNs if there is more than one certificate w/in the same building, where the attribute was populated at least once. Assumes attribute is time-invariant.

TODO: add flag for inputation to be used in the model?

In [15]:
fill_columns = ['built_form',
 'mains_gas_flag',
 'floor_level',
 'number_habitable_rooms',
 'roof_description',
 'heat_loss_corridor',
 'floor_height']

In [16]:
for col in fill_columns:
    
    print(f'Filling {col}')
    
    temp = cert[['building_reference_number','lmk_key',col]].groupby('building_reference_number').count()
    temp = temp[
        (temp.lmk_key>1)& #more than one certificate on the same building
        (temp[col]>0)& #at least one certificate with attribute populated
        (temp[col]<temp.lmk_key) #at least one certificate where attribute missing  
    ]
    
    print(f'Filling in missing values for {len(temp)} properties')
    if len(temp)==0: continue
    
    # Get filler values
    temp_lookup = cert[cert.building_reference_number.isin(temp.index)].groupby('building_reference_number')[col].agg(
        lambda x: pd.Series.mode(x)[0]) #just get the most frequent
    
    #join original table to the lookup table to get the filler values
    temp_filler = cert.loc[cert.building_reference_number.isin(temp.index) & cert[col].isna()].merge(
        temp_lookup, how='left',left_on='building_reference_number',right_index=True)[col+'_y']

    # fill in original table 
    cert.loc[cert.building_reference_number.isin(temp.index) & cert[col].isna(),col
            ] = temp_filler
    

Filling built_form
Filling in missing values for 112 properties
Filling mains_gas_flag
Filling in missing values for 321 properties
Filling floor_level
Filling in missing values for 48 properties
Filling number_habitable_rooms
Filling in missing values for 290 properties
Filling roof_description
Filling in missing values for 24 properties
Filling heat_loss_corridor
Filling in missing values for 132 properties
Filling floor_height
Filling in missing values for 3779 properties


## Dedupe on same date
TODO: didn't do anything with this for now. Consider later.
Quite high % of duplicates (but they could be different apartments? we do not get the apartment number).

In [17]:
len(cert[cert.duplicated(subset=['building_reference_number','inspection_date'])]) / len(cert)

0.022410358565737053

In [18]:
len(cert[cert.duplicated(subset=['uprn','inspection_date'])]) / len(cert)

0.026133560994118763

## Dropping cols that are mostly empty

TODO: Check if worth replicating this.

Not doing it for now because will treat fields differently later on.

## Clipping values

TODO: review all of these thresholds - done using analysis from DSCamp

In [78]:
def call_clip(df):
    
    ''' 
    Clips the variables of the EPC data between 0 and a specified upper amount
    '''
    
    df['current_energy_efficiency'].clip(lower = 0.0, upper = 100, inplace = True)
    df['energy_consumption_current'].clip(lower = 0.0, upper = 600, inplace = True)
    df['co2_emissions_current'].clip(lower = 0.0, upper = 20, inplace = True)
    df['co2_emiss_curr_per_floor_area'].clip(lower = 0.0, upper = 100, inplace = True)
    df['lighting_cost_current'].clip(lower = 0.0, upper = 150, inplace = True)
    df['heating_cost_current'].clip(lower = 0.0, upper = 1750, inplace = True)
    df['hot_water_cost_current'].clip(lower = 0.0, upper = 350, inplace = True)
    df['total_floor_area'].clip(lower = 10.0, upper = 175, inplace = True)
    df['multi_glaze_proportion'].clip(lower = 0.0, upper = 100, inplace = True)
    df['extension_count'].clip(lower = 0.0, upper = 4, inplace = True)
    df['number_habitable_rooms'].clip(lower = 0.0, upper = 10, inplace = True)
    df['number_heated_rooms'].clip(lower = 0.0, upper = 9, inplace = True)
    df['low_energy_lighting'].clip(lower = 0.0, upper = 100, inplace = True)
    df['number_open_fireplaces'].clip(lower = 0.0, upper = 4, inplace = True)
    df['floor_height'].clip(lower = 0.0, upper = 3, inplace = True)
    return(df)

call_clip = call_clip(cert)


## Other checks and cleansing steps

TODO: All this has been taken from DSCamp analysis - evaluate if necessary at all.

#### Remove bi-lingual sentences

In [28]:
descriptive_columns = ['floor_description',
 'lighting_description',
 'roof_description',
 'walls_description',
 'windows_description',
 'hotwater_description',
 'main_heating_controls']

for col in descriptive_columns:
    cert[col] = cert[col].str.replace(r"\|(.*)","")

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\16275547.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  cert[col] = cert[col].str.replace(r"\|(.*)","")


#### Cleaning up units for average thermal transmittance

In [29]:
def thermal_cleanup(df):
    
    ''' 
    standardising the wording in the variables containing average thermal transmittance. Specifically, 
    removing bi-lingual versions, removing =, translating welsh version, and standardising units
    '''
    
    for c in ['floor_description','walls_description','roof_description']:
        df[c] = df[c].str.replace('  ',' ')
        df[c] = df[c].str.replace('Average thermal transmittance 1 ','Average thermal transmittance 1.00 ')
        df[c] = df[c].str.replace('Average thermal transmittance =','Average thermal transmittance')
        df[c] = df[c].str.replace('Trawsyriannedd thermol cyfartalog','Average thermal transmittance')
        # standardising the unit used
        df[c] = df[c].str.replace(r"W(.*?)K",'W/m²K')
        
    return df

cert = thermal_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\4072927915.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df[c] = df[c].str.replace(r"W(.*?)K",'W/m²K')


#### Floor description

In [34]:
def thermal_floor(df):
    
    ''' 
    rounds the average thermal transmittance figure which is within a str field to 1 decimal place
    '''
    
    # finds the decimal number
    df['floors_average_thermal_transmittance'] = df['floor_description'].str.findall(r'\d.\d*')
    # rounds the number to 1 decimal place
    df['floors_average_thermal_transmittance'] = round(df['floors_average_thermal_transmittance'].str[0].astype(float),1)
    # 
    df['floor_description'] = df.apply(lambda row: 'average thermal transmittance %.1f W/m²K' % (row['floors_average_thermal_transmittance']) if 'Average' in str(row['floor_description']) else row['floor_description'],axis=1)
    
    return df

cert = thermal_floor(cert)

In [39]:
def floor_cleanup(df):
    
    ''' 
    cleans-up the floor_description feature from the EPC dataset
    '''
    
    df['floor_description'] = df['floor_description'].str.lower()

    # translating welsh sentences
    df['floor_description'] = df['floor_description'].str.replace(r"anheddiad arall islaw",'another dwelling below') 
    df['floor_description'] = df['floor_description'].str.replace("\(eiddo arall islaw\)",'other premises below') 
    df['floor_description'] = df['floor_description'].str.replace(r"wedi(.*?)i inswleiddio","insulated")
    df['floor_description'] = df['floor_description'].str.replace("dim inswleiddio","no insulation")
    df['floor_description'] = df['floor_description'].str.replace("(rhagdybiaeth)","assumed")
    df['floor_description'] = df['floor_description'].str.replace("crog","suspended")
    df['floor_description'] = df['floor_description'].str.replace("heb ei inswleiddio","no insulation")
    df['floor_description'] = df['floor_description'].str.replace("i ofod heb ei wresogi","to unheated space")
    df['floor_description'] = df['floor_description'].str.replace("solet","solid")
    df['floor_description'] = df['floor_description'].str.replace("inswleiddio cyfyngedig","limited insulation")

    # removing assumed
    df['floor_description'] = df['floor_description'].str.replace("\(assumed\)","")

    # standardising language
    df['floor_description'] = df['floor_description'].str.replace("\(another dwelling below\)",'other premises below') 
    df['floor_description'] = df['floor_description'].str.replace("\(other premises below\)",'other premises below')
    df['floor_description'] = df['floor_description'].str.replace("solid.",'solid,') 
    df['floor_description'] = df['floor_description'].str.replace("uninsulated",'no insulation,') 
    df['floor_description'] = df['floor_description'].str.replace("insulation=100mm",'100 mm insulation') 
    df['floor_description'] = df['floor_description'].str.replace(", \(assumed\)",' (assumed)')
    df['floor_description'] = df['floor_description'].str.replace("insulation=25mm",'25 mm insulation')
    df['floor_description'] = df['floor_description'].str.replace("insulation=75mm",'75 mm insulation')
    df['floor_description'] = df['floor_description'].str.replace("limited insulated",'limited insulation')
    # cleanup
    df['floor_description'] = df['floor_description'].str.replace("[ \t]+$","")
    df['floor_description'] = df['floor_description'].str.replace(r'\,$','')
    df['floor_description'] = df['floor_description'].str.replace(r'^, ','')
    df['floor_description'] = df['floor_description'].str.replace(r'\?','')
    
    return df

cert = floor_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3060261636.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['floor_description'] = df['floor_description'].str.replace("\(eiddo arall islaw\)",'other premises below')
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3060261636.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['floor_description'] = df['floor_description'].str.replace(r"wedi(.*?)i inswleiddio","insulated")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3060261636.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['floor_description'] = df['floor_description'].str.replace("(rhagdybiaeth)","assumed")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3060261636.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  df['floor_description'] = df['floor_description

#### Lighting description

In [52]:
# lighting and second heat descriptions have some entries the wrong way around
cert['secondheat_description1'] = cert.apply(
    lambda row: row['lighting_description'] if 'lighting' in str(row['secondheat_description']) else row['secondheat_description'],axis = 1)
cert['lighting_description1'] = cert.apply(
    lambda row: row['secondheat_description'] if 'lighting' in str(row['secondheat_description']) else row['lighting_description'],axis = 1)

cert.drop(columns= ['secondheat_description','lighting_description'],axis=1,inplace=True)
cert.rename(columns={'secondheat_description1':'secondheat_description','lighting_description1':'lighting_description'},inplace=True)

In [53]:
def lighting_cleanup(df):
    
    ''' 
    cleans-up the lighting_description feature from the EPC dataset
    '''
    
    df['lighting_description'] = df['lighting_description'].str.lower()
    # combining ways of saying all lights are low energy
    df['lighting_description'] = df['lighting_description'].str.replace(r'goleuadau ynni-isel ym mhob un o.r mannau gosod','low energy lighting in all fixed outlets')
    df['lighting_description'] = df['lighting_description'].str.replace(r'low energy lighting in 120% of fixed outlets','low energy lighting in all fixed outlets')
    df['lighting_description'] = df['lighting_description'].str.replace(r'low energy lighting 100% of fixed outlets','low energy lighting in all fixed outlets')
    # translating welsh sentences
    df['lighting_description'] = df['lighting_description'].str.replace(r"\|goleuadau(.*)",'') 
    df['lighting_description'] = df['lighting_description'].str.replace(r"o.r mannau gosod",'of fixed outlets')
    df['lighting_description'] = df['lighting_description'].str.replace('% fixed','% of fixed')
    df['lighting_description'] = df['lighting_description'].str.replace('goleuadau ynni-isel mewn','low energy lighting in')
    df['lighting_description'] = df['lighting_description'].str.replace(r"\.\d*",'')
    df['lighting_description'] = df['lighting_description'].str.replace("dim goleuadau ynni-isel",'no low energy lighting')
    # clean up
    df['lighting_description'] = df['lighting_description'].str.replace("eneregy",'energy')
    
    return df

cert = lighting_cleanup(cert)



C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3090609091.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lighting_description'] = df['lighting_description'].str.replace(r'goleuadau ynni-isel ym mhob un o.r mannau gosod','low energy lighting in all fixed outlets')
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3090609091.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lighting_description'] = df['lighting_description'].str.replace(r"\|goleuadau(.*)",'')
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3090609091.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lighting_description'] = df['lighting_description'].str.replace(r"o.r mannau gosod",'of fixed outlets')
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3090609091.py:17: FutureWarning: The default value of regex will change from True to False in a futur

In [54]:
def lighting_perc_cleanup(df):
    
    ''' 
    rounds the low energy lighting percentage figure which is within a str field
    '''
    
    df['low_energy_lighting_perc'] = df['lighting_description'].str.findall(r'(\d*)\%')
    df['low_energy_lighting_perc'] = round(df['low_energy_lighting_perc'].str[0].astype(float),-1)
    df['lighting_description'] = df.apply(lambda row: 'low energy lighting %d%% of fixed outlets' % (int(row['low_energy_lighting_perc'])) if '%' in str(row['lighting_description']) else row['lighting_description'],axis=1)

    return df

cert = lighting_perc_cleanup(cert)

#### Roof description

In [55]:
def roof_cleanup(df):
    
    ''' 
    cleans-up the roof_description feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'roof_description'
    Returns a dataframe 
    '''
    
    # standardising units
    df['roof_description'] = df['roof_description'].str.replace(" mm",'mm')
    df['roof_description'] = df['roof_description'].str.replace(">= 300mm",">=300mm")
    df['roof_description'] = df['roof_description'].str.replace(">=300mm","300+mm")
    # translating welsh sentences
    df['roof_description'] = df['roof_description'].str.replace("\|\(eiddo arall uwchben\)","")
    df['roof_description'] = df['roof_description'].str.replace("Ar oleddf","Pitched")
    df['roof_description'] = df['roof_description'].str.replace("dim inswleiddio","no insulation")
    df['roof_description'] = df['roof_description'].str.replace(r"wedi(.*?)i inswleiddio","insulated")
    df['roof_description'] = df['roof_description'].str.replace("(rhagdybiaeth)","assumed")
    df['roof_description'] = df['roof_description'].str.replace(r"lo inswleiddio yn y llof.*","loft insulation")
    df['roof_description'] = df['roof_description'].str.replace(r"o inswleiddio yn y llof.*","loft insulation")
    df['roof_description'] = df['roof_description'].str.replace("Ystafell\(oedd\) to","Roof room(s)")
    df['roof_description'] = df['roof_description'].str.replace(r"wedi(.*?)i hinswleiddio","insulated")
    df['roof_description'] = df['roof_description'].str.replace("nenfwd","ceiling")
    df['roof_description'] = df['roof_description'].str.replace("wrth y trawstia(.*?)","at rafters")
    df['roof_description'] = df['roof_description'].str.replace("inswleiddio cyfyngedig","limited insulation")
    df['roof_description'] = df['roof_description'].str.replace("To gwellt, gydag inswleiddio ychwanegol","Thatched, with additional insulation")
    df['roof_description'] = df['roof_description'].str.replace("Yn wastad","Always")
    # standardising descriptions
    df['roof_description'] = df['roof_description'].str.replace("Roof room,","Roof room(s),")
    df['roof_description'] = df['roof_description'].str.replace("annedd arall uwchben","other premises above")
    df['roof_description'] = df['roof_description'].str.replace("another dwelling above","other premises above")
    df['roof_description'] = df['roof_description'].str.replace("Dwelling Above","(other premises above)")
    df['roof_description'] = df['roof_description'].str.replace(r"0 W/m²K",r" W/m²K")
    df['roof_description'] = df['roof_description'].str.replace("  W/m²K"," 0.0 W/m²K")
    df['roof_description'] = df['roof_description'].str.replace("Roof room\(s\), no insulation\(assumed\)","Roof room(s), no insulation (assumed)")
    df['roof_description'] = df['roof_description'].str.replace("Other premises above","(other premises above)")
    df['roof_description'] = df['roof_description'].str.replace("\(assumed\)","")
    # cleanup
    df['roof_description'] = df['roof_description'].str.replace(r'\.$','')
    df['roof_description'] = df['roof_description'].str.replace(r'\,$','')
    df['roof_description'] = df['roof_description'].str.replace("  \+"," +")
    df['roof_description'] = df['roof_description'].str.replace("[ \t]+$","")
    df['roof_description'] = df['roof_description'].str.replace("mmmm","mm")
    df['roof_description'] = df['roof_description'].str.replace("Thatchedinsulated","Thatched, insulated")
    df['roof_description'] = df['roof_description'].str.replace("\*\*\* INVALID INPUT Code \: 57 \*\*\*","")
    
    return df

cert = roof_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3932964614.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['roof_description'] = df['roof_description'].str.replace("\|\(eiddo arall uwchben\)","")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3932964614.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['roof_description'] = df['roof_description'].str.replace(r"wedi(.*?)i inswleiddio","insulated")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3932964614.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['roof_description'] = df['roof_description'].str.replace("(rhagdybiaeth)","assumed")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3932964614.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['roof_description'] = df['roof_description'].str.replace(r"lo insw

In [56]:
def thermal_roof(df):
    
    ''' 
    rounds the average thermal transmittance figure which is within a str field
    '''
    
    df['roof_average_thermal_transmittance'] = df['roof_description'].str.findall(r'(\d.\d*) W/m²K')
    df['roof_average_thermal_transmittance'] = round(df['roof_average_thermal_transmittance'].str[0].astype(float),1)
    df['roof_description'] = df.apply(lambda row: 'average thermal transmittance %.1f w/m²k' % (row['roof_average_thermal_transmittance']) if 'Average' in str(row['roof_description']) else row['roof_description'],axis=1)
    
    return df

cert = thermal_roof(cert)

#### Walls description

In [57]:
def walls_cleanup(df): 
    
    ''' 
    cleans-up the walls_description feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'walls_description'
    Returns a dataframe 
    '''
    
    # removing phrases like as built which aren't adding anything
    df['walls_description'] = df['walls_description'].str.lower()
    df['walls_description'] = df['walls_description'].str.replace(r"fel y(.*?)u hadeiladwyd, ","")
    df['walls_description'] = df['walls_description'].str.replace("as built, ","")
    df['walls_description'] = df['walls_description'].str.replace("cavity\.","cavity wall,")
    df['walls_description'] = df['walls_description'].str.replace("\(rhagdybiaeth\)","(assumed)")
    df['walls_description'] = df['walls_description'].str.replace("\(assumed\)","")
    # translating welsh sentences
    df['walls_description'] = df['walls_description'].str.replace("waliau ceudod","cavity wall")
    df['walls_description'] = df['walls_description'].str.replace("dim inswleiddio","no insulation")
    df['walls_description'] = df['walls_description'].str.replace("tywodfaen","sandstone")
    df['walls_description'] = df['walls_description'].str.replace("ceudod wedi(.*?)i lenwi","filled cavity")
    df['walls_description'] = df['walls_description'].str.replace(r"wedi(.*?)u hinswleiddio","insulated")
    df['walls_description'] = df['walls_description'].str.replace(r"ffr(.*?)m bren","timber frame")
    df['walls_description'] = df['walls_description'].str.replace("briciau solet","solid brick")
    df['walls_description'] = df['walls_description'].str.replace("wedi(.*?)u hadeiladu yn (.*?)l system","system built")
    df['walls_description'] = df['walls_description'].str.replace("inswleiddio rhannol","partial insulation")
    df['walls_description'] = df['walls_description'].str.replace("gydag inswleiddio allanol","with external insulation")
    df['walls_description'] = df['walls_description'].str.replace("gwenithfaen neu risgraig","granite or whinstone")
    df['walls_description'] = df['walls_description'].str.replace("gydag inswleiddio mewnol","with internal insulation")
    # standardising punctuation
    df['walls_description'] = df['walls_description'].str.replace("solid brick\.","solid brick,")
    # standardising language
    df['walls_description'] = df['walls_description'].str.replace("granite or whin,","granite or whinstone,")
    df['walls_description'] = df['walls_description'].str.replace("stone \(granite or whin\)\.","granite or whinstone,")
    df['walls_description'] = df['walls_description'].str.replace("with external insulation","insulated")
    df['walls_description'] = df['walls_description'].str.replace("with internal insulation","insulated")
    df['walls_description'] = df['walls_description'].str.replace("with additional insulation","insulated")
    df['walls_description'] = df['walls_description'].str.replace("with insulation","insulated")
    # clean up
    df['walls_description'] = df['walls_description'].str.replace(r'\.$','')
    df['walls_description'] = df['walls_description'].str.replace('\+ chr\(13\) \+','+')
    df['walls_description'] = df['walls_description'].str.replace("  \+"," +")
    df['walls_description'] = df['walls_description'].str.replace("[ \t]+$","")
    df['walls_description'] = df['walls_description'].str.replace('timber frame\.','timber frame,')
    df['walls_description'] = df['walls_description'].str.replace('\?','')
    df['walls_description'] = df['walls_description'].str.replace('system built\.','system built,')
    
    return df

cert = walls_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2908878170.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['walls_description'] = df['walls_description'].str.replace(r"fel y(.*?)u hadeiladwyd, ","")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2908878170.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['walls_description'] = df['walls_description'].str.replace("cavity\.","cavity wall,")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2908878170.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['walls_description'] = df['walls_description'].str.replace("\(rhagdybiaeth\)","(assumed)")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2908878170.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['walls_description'] = df['walls_description'].str.replace("\(assum

In [58]:
def thermal_walls(df):
    
    ''' 
    rounds the average thermal transmittance figure which is within a str field
    '''
    
    df['walls_average_thermal_transmittance'] = df['walls_description'].str.findall(r'\d.\d\d')
    df['walls_average_thermal_transmittance'] = round(df['walls_average_thermal_transmittance'].str[0].astype(float),1)
    df['walls_description'] = df.apply(lambda row: 'average thermal transmittance %.1f w/m²k' % (row['walls_average_thermal_transmittance']) if 'average' in str(row['walls_description']) else row['walls_description'],axis=1)
    
    return df

cert = thermal_walls(cert)

#### Windows description

In [59]:
def windows_cleanup(df): 
    
    ''' 
    cleans-up the windows_description feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'windows_description'
    Returns a dataframe 
    '''
    
    df['windows_description'] = df['windows_description'].str.lower()
    # translating welsh sentences
    df['windows_description'] = df['windows_description'].str.replace("ffenestri perfformiad uchel","high performance glazing")
    df['windows_description'] = df['windows_description'].str.replace("gwydrau dwbl gan mwyaf","mostly double glazing")
    df['windows_description'] = df['windows_description'].str.replace("rhai gwydrau dwbl","partial double glazing")
    df['windows_description'] = df['windows_description'].str.replace("gwydrau sengl","single glazing")
    df['windows_description'] = df['windows_description'].str.replace("gwydrau dwbl rhannol","partial double glazing")
    df['windows_description'] = df['windows_description'].str.replace("gwydrau dwbl llawn","fully double glazing")
    df['windows_description'] = df['windows_description'].str.replace("gwydrau lluosog ym mhobman","multiple glazing throughout")
    df['windows_description'] = df['windows_description'].str.replace("gwydrau eilaidd llawn","full secondary glazing")
    # standardising language
    df['windows_description'] = df['windows_description'].str.replace("glazed","glazing")
    df['windows_description'] = df['windows_description'].str.replace("fully","full")
    df['windows_description'] = df['windows_description'].str.replace("fully","full")
    # cleanup
    df['windows_description'] = df['windows_description'].str.replace("single glazingsingle glazing","single glazing")
    df['windows_description'] = df['windows_description'].str.replace("single glazingdouble glazing","single glazing and double glazing")
    df['windows_description'] = df['windows_description'].str.replace("single glazingsecondary glazing","single glazing and secondary glazing")
    df['windows_description'] = df['windows_description'].str.replace("[ \t]+$","")
    df['windows_description'] = df['windows_description'].str.replace("  "," ")
    
    return df

cert = windows_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3119369361.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['windows_description'] = df['windows_description'].str.replace("[ \t]+$","")


#### Hotwater description

In [60]:
def hotwater_cleanup(df):
    
    ''' 
    cleans-up the hotwater_description feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'hotwater_description'
    Returns a dataframe 
    '''
    
    # removing spaces at the end
    df['hotwater_description'] = df['hotwater_description'].str.replace("[ \t]+$","")

    # translating welsh sentences
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"O(.*?)r brif system","From main system")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"Trochi trydan","Electric immersion")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"an-frig","off-peak")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"O system eilaidd","From secondary system")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"Nwy wrth fwy nag un pwynt","Gas multipoint")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"Popty estynedig olew","Oil range cooker")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"tarriff safonol","standard tariff")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"Dim system ar gael","No system present")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"adfer gwres nwyon ffliw","flue gas heat recovery")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"gydag ynni(.*?)r haul","plus solar")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"dim thermostat ar y silindr","no cylinderstat")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"rhagdybir bod twymwr tanddwr trydan","electric immersion assumed")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"an-frig","off peak")
    df['hotwater_description'] = df['hotwater_description'].str.replace(r"Twymwr tanddwr","underfloor heating")

    # standardising language
    df['hotwater_description'] = df['hotwater_description'].str.replace("cylinder thermostat","cylinderstat")
    df['hotwater_description'] = df['hotwater_description'].str.replace("No system present :","No system present:")
    df['hotwater_description'] = df['hotwater_description'].str.replace("No hot water system present -","No system present:")
    df['hotwater_description'] = df['hotwater_description'].str.replace("From community scheme","Community scheme")
    df['hotwater_description'] = df['hotwater_description'].str.replace("Community heat pump","Community scheme with CHP")
    df['hotwater_description'] = df['hotwater_description'].str.replace("From secondary heater","From secondary system")
    df['hotwater_description'] = df['hotwater_description'].str.replace("SAP05:Hot-Water","SAP:Hot-Water")
    df['hotwater_description'] = df['hotwater_description'].str.replace("community scheme","Community scheme")
    df['hotwater_description'] = df['hotwater_description'].str.replace("plus solar, no cylinderstat","no cylinderstat, plus solar")
    df['hotwater_description'] = df['hotwater_description'].str.replace("From second main heating system","From secondary system")
    df['hotwater_description'] = df['hotwater_description'].str.replace("none","No system present: electric immersion assumed")
    df['hotwater_description'] = df['hotwater_description'].str.replace("plus solar, flue gas heat recovery","flue gas heat recovery, plus solar")
    df['hotwater_description'] = df['hotwater_description'].str.replace("no cylinderstat, no cylinderstat","no cylinderstat")

    # cleanup
    df['hotwater_description'] = df['hotwater_description'].str.replace("No system present\?electric immersion assumed","No system present: electric immersion assumed")
    df['hotwater_description'] = df['hotwater_description'].replace("***SAMPLE***",np.nan)

    return df

cert = hotwater_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3908772556.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hotwater_description'] = df['hotwater_description'].str.replace("[ \t]+$","")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3908772556.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hotwater_description'] = df['hotwater_description'].str.replace(r"O(.*?)r brif system","From main system")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3908772556.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hotwater_description'] = df['hotwater_description'].str.replace(r"gydag ynni(.*?)r haul","plus solar")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\3908772556.py:45: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hotwater_description'] = df['hotwater_descri

#### Main heating controls

In [61]:
def heat_control_cleanup(df):
    
    ''' 
    cleans-up the main_heating_controls feature from the EPC dataset
    Parameters
      df: a dataframe containing a variable called 'main_heating_controls'
    Returns a dataframe 
    '''
    
    df['main_heating_controls'] = df['main_heating_controls'].str.lower()

    # translating welsh sentences
    df['main_heating_controls'] = df['main_heating_controls'].str.replace(r"rheoli.r t.l . llaw","manual charge control")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rhaglennydd, dim thermostat ystafell","programmer, no room thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rheolaeth amser a rheolaeth parthau tymheredd","time and temperature zone control")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rhaglennydd a thermostat ystafell","programmer and room thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rhaglennydd a thermostatau ar y cyfarpar","programmer and appliance thermostats")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rhaglennydd ac o leiaf ddau thermostat ystafell","programmer and at least two room thermostats")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("thermostat ystafell yn unig","room thermostat only")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("dim rheolaeth thermostatig ar dymheredd yr ystafell","no thermostatic control of room temperature")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rheoli gwefr drydanol yn awtomatig","automatic charge control")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("dim rheolaeth amser na rheolaeth thermostatig ar dymheredd yr ystafell","no time or thermostatic control of room temperature")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("trvs a falf osgoi","trvs and bypass")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("rhaglennydd","programmer")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("tal un gyfradd, thermostat ystafell yn unig","flat rate charging, room thermostat only")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("thermostat ystafell a trvs","room thermostat and trvs")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("t(.*?)l un gyfradd","flat rate charging")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("thermostatau ar y cyfarpar","appliance thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("dim","none")

    # standardising language
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmer, no thermostat","programmer, no room thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("flat rate charging\*","flat rate charging")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("\+","and")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("\&","and")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("trv.s","trvs")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("thermostats","thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("communit ","community ")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("to the use of community heating","to use of community heating")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace(" stat"," thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("controls","control")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("prog ","programmer ")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("program ","programmer ")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmerand","programmer and")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("delayed start thermostat and program and trvs","delayed start thermostat, program and trvs")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("thermostatic","thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("flat rate charging, programmer no room thermostat","flat rate charging, programmer, no room thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace(" 2 "," two ")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("roomstat","room thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmer and room thermostat and trvs","programmer, room thermostat and trvs")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmer and trvs and boiler energy manager","programmer, trvs and boiler energy manager")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmer and trvs and bypass","programmer, trvs and bypass")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmer and trvs and flow switch","programmer, trvs and flow switch")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("temp+$","temperature")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("no thermostat control of room temperature","no thermostat control")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("appliance thermostat and programmer","programmer and appliance thermostat")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("delayed start thermostat and programmer and trvs","delayed start thermostat, programmer and trvs")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("no time or thermostat control of temperature","no time or thermostat control of room temperature")
    df['main_heating_controls'] = df['main_heating_controls'].str.replace("programmer\?and","programmer and")
  
    
    return df

cert = heat_control_cleanup(cert)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2782498958.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['main_heating_controls'] = df['main_heating_controls'].str.replace(r"rheoli.r t.l . llaw","manual charge control")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2782498958.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df['main_heating_controls'] = df['main_heating_controls'].str.replace("t(.*?)l un gyfradd","flat rate charging")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2782498958.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['main_heating_controls'] = df['main_heating_controls'].str.replace("flat rate charging\*","flat rate charging")
C:\Users\Joanna\AppData\Local\Temp\ipykernel_796\2782498958.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  df

In [63]:
# this might have been dealt with already 
cert['transaction_type'] = cert['transaction_type'].str.replace(" - this is for backwards compatibility only and should not be used","")
cert['glazed_type'] = cert['glazed_type'].replace('INVALID!',np.nan)

In [65]:
cert.head()

,lmk_key,address1,address2,address3,postcode,building_reference_number,current_energy_rating,potential_energy_rating,current_energy_efficiency,potential_energy_efficiency,property_type,built_form,inspection_date,local_authority,constituency,county,lodgement_date,transaction_type,environment_impact_current,environment_impact_potential,energy_consumption_current,energy_consumption_potential,co2_emissions_current,co2_emiss_curr_per_floor_area,co2_emissions_potential,lighting_cost_current,lighting_cost_potential,heating_cost_current,heating_cost_potential,hot_water_cost_current,hot_water_cost_potential,total_floor_area,energy_tariff,mains_gas_flag,floor_level,flat_top_storey,flat_storey_count,main_heating_controls,multi_glaze_proportion,glazed_type,glazed_area,extension_count,number_habitable_rooms,number_heated_rooms,low_energy_lighting,number_open_fireplaces,hotwater_description,hot_water_energy_eff,hot_water_env_eff,floor_description,floor_energy_eff,floor_env_eff,windows_description,windows_energy_eff,windows_env_eff,walls_description,walls_energy_eff,walls_env_eff,sheating_energy_eff,sheating_env_eff,roof_description,roof_energy_eff,roof_env_eff,mainheat_description,mainheat_energy_eff,mainheat_env_eff,mainheatcont_description,mainheatc_energy_eff,mainheatc_env_eff,lighting_energy_eff,lighting_env_eff,main_fuel,wind_turbine_count,heat_loss_corridor,unheated_corridor_length,floor_height,photo_supply,solar_water_heating_flag,mechanical_ventilation,address,local_authority_label,constituency_label,posttown,construction_age_band,lodgement_datetime,tenure,fixed_lighting_outlets_count,low_energy_fixed_light_count,uprn,uprn_source,floors_average_thermal_transmittance,secondheat_description,lighting_description,low_energy_lighting_perc,roof_average_thermal_transmittance,walls_average_thermal_transmittance
0,515555893032010071912500461968805,"51, Heathfield Drive",NaN,NaN,TS25 5LW,9551808768,C,C,71,73,House,Semi-Detached,2010-07-19,E06000001,E14000733,NaN,2010-07-19,marketed sale,68,69,229.000,221.000,2.900,38.000,2.800,76.000,41.000,457.000,463.000,95.000,95.000,77.040,Single,Y,NaN,NaN,NaN,NaN,100.000,double glazing installed during or after 2002,Normal,1.000,5.000,5.000,12.000,0.000,From main system,Very Good,Very Good,"suspended, no insulation",NaN,NaN,full double glazing,Good,Good,"cavity wall, filled cavity",Good,Good,NaN,NaN,"Pitched, 100mm loft insulation",Average,Average,"Boiler and radiators, mains gas",Very Good,Very Good,"Programmer, room thermostat and TRVs",Good,Good,Poor,Poor,mains gas - this is for backwards compatibilit...,0.000,NaN,NaN,2.640,0.000,N,natural,"51, Heathfield Drive",Hartlepool,Hartlepool,HARTLEPOOL,England and Wales: 1930-1949,2010-07-19 12:50:04,owner-occupied,NaN,NaN,100110014445.000,Address Matched,NaN,None,low energy lighting 10% of fixed outlets,10.000,NaN,NaN
1,759491544532015040713013761068507,"37, Waverley Terrace",NaN,NaN,TS25 5NB,4701526968,E,B,54,83,House,Semi-Detached,2015-04-07,E06000001,E14000733,NaN,2015-04-07,non marketed sale,45,79,335.000,124.000,7.300,59.000,2.700,138.000,69.000,1317.000,698.000,119.000,80.000,124.000,Single,Y,NaN,NaN,NaN,NaN,100.000,double glazing installed before 2002,Normal,0.000,6.000,6.000,0.000,0.000,From main system,Good,Good,"suspended, no insulation",NaN,NaN,full double glazing,Average,Average,"cavity wall, no insulation",Poor,Poor,NaN,NaN,"Pitched, 12mm loft insulation",Very Poor,Very Poor,"Boiler and radiators, mains gas",Good,Good,"Programmer, no room thermostat",Very Poor,Very Poor,Very Poor,Very Poor,mains gas (not community),0.000,NaN,NaN,2.510,NaN,N,natural,"37, Waverley Terrace",Hartlepool,Hartlepool,HARTLEPOOL,England and Wales: 1930-1949,2015-04-07 13:01:37,rental (private),NaN,NaN,100110033672.000,Address Matched,NaN,None,no low energy lighting,NaN,NaN,NaN
2,743455119942012012615204198522168,"14, Carlisle Street",NaN,NaN,TS25 1BL,955115968,E,D,47,61,House,Mid-Terrace,2012-01-26,E06000001,E14000733,NaN,2012-01-26,marketed sale,42,56,326.000,234.000,7.

## Further work on categorical vars

#### Floor level
Will create 2 variables - a variable with the floor number (where available), and another one with some rough classes (bottom, mid, top floors)

In [90]:
#TODO: review these classes when dataset is complete
floor_level_numeric_mapper = {
    '1st':'1',
    'Ground':'0',
    '2nd':'2',
    '01':'1',
    '00':'0',
    '3rd':'3',
    '02':'2',
    'ground floor':'0',
    '1.0':'1',
    '0.0':'0',
    '2.0':'2',
    '4th':'4',
    '03':'3',
    '3':'3',
    '1':'1',
    '2':'2',
    '0':'0',
    '-1':'-1',
    'Basement':'-1'
}

cert['floor_level_numeric'] = cert.floor_level.map(floor_level_numeric_mapper)

cert.floor_level_numeric.value_counts()

1     3140
0     2741
2     1105
3      365
4       47
-1      11
Name: floor_level_numeric, dtype: int64

In [98]:
# review these when dataset is complete
floor_level_dict = dict.fromkeys(['Ground','ground floor','Basement',],'ground floor')
floor_level_dict1 = dict.fromkeys(['1st','2nd','3rd','4th'],'low floors')
floor_level_dict2 = dict.fromkeys(['mid floor','5th','6th','7th','8th','9th','10th','11th'],'mid floors')
floor_level_dict3 = dict.fromkeys(['top floor','12th','13th','14th','15th','16th','17th','18th','19th','20th',
                                   '21st or above'],'top floors')
floor_level_dict4 = {
    '1st':'low floors',
    'Ground':'ground floor',
    '2nd':'low floors',
    '01':'low floors',
    '00':'ground floor',
    '3rd':'low floors',
    '02':'low floors',
    'ground floor':'ground floor',
    '1.0':'low floors',
    '0.0':'ground floor',
    '2.0':'low floors',
    '4th':'low floors',
    '03':'low floors',
    '3':'low floors',
    '1':'low floors',
    '2':'low floors',
    '0':'ground floor',
    '-1':'ground floor',
    'Basement':'ground floor'
}

floor_level_dict.update(floor_level_dict1)
floor_level_dict.update(floor_level_dict2)
floor_level_dict.update(floor_level_dict3)
floor_level_dict.update(floor_level_dict4)

cert['floor_level_classes'] = cert.floor_level.map(floor_level_dict)

cert.floor_level_classes.value_counts()



low floors      4657
ground floor    2752
mid floors       188
top floors       129
Name: floor_level_classes, dtype: int64

In [99]:
# cert.floor_level_classes.isna().sum(), cert.floor_level_numeric.isna().sum()

(34442, 34759)

## Notes

#### Vars to include
'property_type', 
'built_form', 
'total_floor_area', 
'energy_tariff', 
'mains_gas_flag',
'floor_level_numeric',

#### To consider as user-inputs
All costs variables (lighting, heting, hot water etc.). Try to understand how they are calculated and if they can be derived from bills - if so they can be input by user



In [86]:
cert.floor_level.dtype

dtype('O')

In [84]:
cert.floor_level.isna().sum()

34156

In [88]:
floor_level_numeric_mapper = {
    '1st':'1',
    'Ground':'0',
    '2nd':'2',
    '01':'1',
    '00':'0',
    '3rd':'3',
    '02':'2',
    'ground floor':'0',
    '1.0':'1',
    '0.0':'0',
    '2.0':'2',
    '4th':'4',
    '03':'3',
    '3':'3',
    '1':'1',
    '2':'2',
    '0':'0',
    '-1':'-1',
    'Basement':'-1'
}

cert.floor_level.map(floor_level_numeric_mapper).value_counts()

1     3140
0     2741
2     1105
3      365
4       47
-1       2
Name: floor_level, dtype: int64

In [85]:
cert.floor_level.value_counts()

1st             2711
Ground          2251
2nd              949
01               401
00               367
3rd              291
mid floor        172
02               152
top floor        129
ground floor     123
1.0              110
0.0              107
2.0               48
4th               47
03                45
3                 29
1                 28
5th               16
04                 9
Basement           9
3.0                8
2                  4
05                 3
-1                 2
4.0                1
Name: floor_level, dtype: int64

In [66]:
cert.head()

,lmk_key,address1,address2,address3,postcode,building_reference_number,current_energy_rating,potential_energy_rating,current_energy_efficiency,potential_energy_efficiency,property_type,built_form,inspection_date,local_authority,constituency,county,lodgement_date,transaction_type,environment_impact_current,environment_impact_potential,energy_consumption_current,energy_consumption_potential,co2_emissions_current,co2_emiss_curr_per_floor_area,co2_emissions_potential,lighting_cost_current,lighting_cost_potential,heating_cost_current,heating_cost_potential,hot_water_cost_current,hot_water_cost_potential,total_floor_area,energy_tariff,mains_gas_flag,floor_level,flat_top_storey,flat_storey_count,main_heating_controls,multi_glaze_proportion,glazed_type,glazed_area,extension_count,number_habitable_rooms,number_heated_rooms,low_energy_lighting,number_open_fireplaces,hotwater_description,hot_water_energy_eff,hot_water_env_eff,floor_description,floor_energy_eff,floor_env_eff,windows_description,windows_energy_eff,windows_env_eff,walls_description,walls_energy_eff,walls_env_eff,sheating_energy_eff,sheating_env_eff,roof_description,roof_energy_eff,roof_env_eff,mainheat_description,mainheat_energy_eff,mainheat_env_eff,mainheatcont_description,mainheatc_energy_eff,mainheatc_env_eff,lighting_energy_eff,lighting_env_eff,main_fuel,wind_turbine_count,heat_loss_corridor,unheated_corridor_length,floor_height,photo_supply,solar_water_heating_flag,mechanical_ventilation,address,local_authority_label,constituency_label,posttown,construction_age_band,lodgement_datetime,tenure,fixed_lighting_outlets_count,low_energy_fixed_light_count,uprn,uprn_source,floors_average_thermal_transmittance,secondheat_description,lighting_description,low_energy_lighting_perc,roof_average_thermal_transmittance,walls_average_thermal_transmittance
0,515555893032010071912500461968805,"51, Heathfield Drive",NaN,NaN,TS25 5LW,9551808768,C,C,71,73,House,Semi-Detached,2010-07-19,E06000001,E14000733,NaN,2010-07-19,marketed sale,68,69,229.000,221.000,2.900,38.000,2.800,76.000,41.000,457.000,463.000,95.000,95.000,77.040,Single,Y,NaN,NaN,NaN,NaN,100.000,double glazing installed during or after 2002,Normal,1.000,5.000,5.000,12.000,0.000,From main system,Very Good,Very Good,"suspended, no insulation",NaN,NaN,full double glazing,Good,Good,"cavity wall, filled cavity",Good,Good,NaN,NaN,"Pitched, 100mm loft insulation",Average,Average,"Boiler and radiators, mains gas",Very Good,Very Good,"Programmer, room thermostat and TRVs",Good,Good,Poor,Poor,mains gas - this is for backwards compatibilit...,0.000,NaN,NaN,2.640,0.000,N,natural,"51, Heathfield Drive",Hartlepool,Hartlepool,HARTLEPOOL,England and Wales: 1930-1949,2010-07-19 12:50:04,owner-occupied,NaN,NaN,100110014445.000,Address Matched,NaN,None,low energy lighting 10% of fixed outlets,10.000,NaN,NaN
1,759491544532015040713013761068507,"37, Waverley Terrace",NaN,NaN,TS25 5NB,4701526968,E,B,54,83,House,Semi-Detached,2015-04-07,E06000001,E14000733,NaN,2015-04-07,non marketed sale,45,79,335.000,124.000,7.300,59.000,2.700,138.000,69.000,1317.000,698.000,119.000,80.000,124.000,Single,Y,NaN,NaN,NaN,NaN,100.000,double glazing installed before 2002,Normal,0.000,6.000,6.000,0.000,0.000,From main system,Good,Good,"suspended, no insulation",NaN,NaN,full double glazing,Average,Average,"cavity wall, no insulation",Poor,Poor,NaN,NaN,"Pitched, 12mm loft insulation",Very Poor,Very Poor,"Boiler and radiators, mains gas",Good,Good,"Programmer, no room thermostat",Very Poor,Very Poor,Very Poor,Very Poor,mains gas (not community),0.000,NaN,NaN,2.510,NaN,N,natural,"37, Waverley Terrace",Hartlepool,Hartlepool,HARTLEPOOL,England and Wales: 1930-1949,2015-04-07 13:01:37,rental (private),NaN,NaN,100110033672.000,Address Matched,NaN,None,no low energy lighting,NaN,NaN,NaN
2,743455119942012012615204198522168,"14, Carlisle Street",NaN,NaN,TS25 1BL,955115968,E,D,47,61,House,Mid-Terrace,2012-01-26,E06000001,E14000733,NaN,2012-01-26,marketed sale,42,56,326.000,234.000,7.

In [103]:
df_rec[df_rec.IMPROVEMENT_ID == 35]

,LMK_KEY,IMPROVEMENT_ITEM,IMPROVEMENT_SUMMARY_TEXT,IMPROVEMENT_DESCR_TEXT,IMPROVEMENT_ID,IMPROVEMENT_ID_TEXT,INDICATIVE_COST
0,515555893032010071912500461968805,1,NaN,NaN,35.000,Low energy lighting for all fixed outlets,NaN
6,759491544532015040713013761068507,4,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£60
10,743455119942012012615204198522168,1,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£8
29,1247581905852014121020132898049031,3,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£50
34,619074199542011041810005282599398,1,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£30
...,...,...,...,...,...,...,...
177221,0400d254aa7c7196051d3c8cb870d9706253fc29e4085c...,2,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£35
177230,056b5d0aa3e1c90570c30a6e838459c19787c22ee1f0f5...,3,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£15
177236,05749930ebd0db33fb7bf20541a31ea892f1877008e837...,4,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£30
177242,058fb621e390c6c76fd2212e765a87e3940e3eecdcd0ac...,3,NaN,NaN,35.000,Low energy lighting for all fixed outlets,£90
